In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib agg

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
    
import sys, os
sys.path.append(os.path.expanduser("~/Jupyter/HPIB/modules"))    
from INOSerial import Arduino
from HPIB4155 import HP4155
from HPIB_plot import Plot, PlotVgs, PlotVp, CalcIsSat, SecDer, Plot2P
from DevParams import UMC

sys.path.append(os.path.expanduser("~/Jupyter/BFmodule"))
sys.path.append(os.path.expanduser("~/Jupyter/BFmodule/bin"))
from BFmodule import DR

sys.path.append(os.path.expanduser("~/Jupyter/YFunc"))
from yfunc import YFuncExtraction

def WriteLog(msg, path, mode='a', end='\n', output=True):
    with open(path, mode) as logfile:
        logfile.write(msg+end)
    if output:
        print(f"{msg}{end}", end='')

from IPython.display import clear_output, display
from os import makedirs, rename
from time import sleep
from datetime import datetime, timedelta

HP=HP4155("GPIB0::17", debug=False)
HP.SetIntTime("LONG")

INO=Arduino("COM3")
print(INO.ask('*'.encode()))

Elsa=DR()

HEWLETT-PACKARD,4155A,0,01.04:01.04:01.00
InoMatrix

Elsa v2.4.2


In [19]:
def TestDevice(device, chn, path, params):
    global HP, INO, Elsa, prog_bar
    print(INO.opench(chn+1))

    if not device:
        print(INO.opench(0))
        sleep(2)
        return 0

    if device[:2] not in ['CA', 'CB', 'TP', 'TN', 'DP', 'DN']:
        return "Invalid device"

    WriteLog(f"## Ch {chn+1} {device}", path + 'log.txt')
    pathp=path+device
    makedirs(pathp, exist_ok=True)
    
    ####################### Measure Diode
    #
    #
    if 'd' in device.lower():
        
        HP.SetStop("COMP")
        HP.SetIntTime("MED")
        HP.SingleDiode(params['Vfmin'], params['Vfmax'], params['Vfstep'], 'SMU2', 'SMU1', Comp=params['IComp'])
            
        now=datetime.now().strftime('%y%m%d %H%M%S')
        Plot(HP.SingleSave(f"{pathp}/{now}.csv", timeout=30, real=True), 'Vf', 'If')
        rename(f"{pathp}/{now}.csv", f"{pathp}/{format(Elsa.GetT('t4k'), '07.3f')} - {now}.csv")
        rename(f"{pathp}/{now}.png", f"{pathp}/{format(Elsa.GetT('t4k'), '07.3f')} - {now}.png")
        
        now=datetime.now()
        while True:
            if (datetime.now()-now).seconds > params['min_wait']: break
            prog_bar.update(f"{datetime.now().strftime('%H:%M:%S')} Waiting minimum {params['min_wait']}s")
            sleep(0.5)
            
        HP.SetStop("OFF")
        HP.SetIntTime("LONG")
        
        print(INO.opench(0))
        WriteLog('', path + 'log.txt')
        return 0
    #
    #
    #######################

    ####################### Measure Transistor
    #
    #

    if 't' in device.lower():
        
        ptype='p' in device.lower()
        
        HP.SetVgs(params['Vmin'], params['Vmax'], params['Vstep'], params['Vd'], ptype=ptype)
  
        now=datetime.now().strftime('%y%m%d %H%M')
        LIN = PlotVgs(HP.SingleSave(f"{pathp}/IdVgs - {now}.csv", timeout=30))
        temp=format(Elsa.GetT('t4k'), '07.3f')
        newname=f"{pathp}/IdVgs - {temp} - {now}"
        rename(f"{pathp}/IdVgs - {now}.csv", f"{newname}.csv")
        rename(f"{pathp}/IdVgs - {now}.png", f"{newname}.png")
        now=datetime.now()
            
        WriteLog(f"Vth={LIN} V", path + 'log.txt')
        try:
            DGM=SecDer(f"{newname}.csv")
            WriteLog(f"SecDer={format(DGM, '.2f')} V", path + 'log.txt')
        except:
            WriteLog(f"SecDer Fail", path + 'log.txt')

        try:
            Vth, SS, miyf, theta1, theta2 = YFuncExtraction(f"{newname}.csv", UMC[int(device[2:])], 4.2, 3.9, params['Vd'])
            WriteLog(f"Yfunc \nVth={format(Vth, '.2f')} V, SS={format(SS*1e3, '.1f')} mV/dec, miyf={format(miyf, '.1f')}, theta1={format(theta1, '.3e')}, theta2={format(theta2, '.3e')}", path + 'log.txt')
            WriteLog(f"{temp},{format(Vth, '.2f')},{format(SS*1e3, '.1f')},{format(miyf, '.1f')},{format(theta1, '.3e')},{format(theta2, '.3e')}", pathp + '/params.txt')
        except:
            WriteLog(f"YFunc Fail", path + 'log.txt')
        
        while True:
            if (datetime.now()-now).seconds > params['min_wait']: break
            prog_bar.update(f"{datetime.now().strftime('%H:%M:%S')} Waiting minimum {params['min_wait']}s")
            sleep(0.5)
            
        if params['HiPot']:
            HP.SetVgs(params['Vmin'], params['Vmax'], params['Vstep'], params['Vmax'], ptype=ptype, sat=True)
            n, Ispec = CalcIsSat(HP.SingleSave(f"{pathp}/IdVgsSat - {now}.csv", timeout=30), Elsa.GetT('t4k'))
            rename(f"{pathp}/IdVgsSat - {now}.csv", f"{pathp}/IdVgsSat - {format(Elsa.GetT('t4k'), '07.3f')} - {now}.csv")
            WriteLog(f"n={format(n, '.3f')}, Ispec={format(Ispec, '.3e')} A", path + 'log.txt')
            sleep(60)
            
            if Ispec != 0:
                HP.SetVp(Ispec, params['Vmin'], params['Vmax'], 0.05, ptype=ptype)
                VTO=PlotVp(HP.SingleSave(f"{pathp}/VpVg - {now}.csv", timeout=30))
                rename(f"{pathp}/VpVg - {now}.csv", f"{pathp}/VpVg - {format(Elsa.GetT('t4k'), '07.3f')} - {now}.csv")
                WriteLog(f"VTO= {VTO} V", path + 'log.txt')
                sleep(60)
                
            HP.SetVds(params['Vmin'], params['Vmax'], params['Vstep'], params['Vgmin'], params['Vgmax'], params['Vgstep'], ptype=ptype)
            Plot(HP.SingleSave(f"{pathp}/IdVds - {now}.csv", timeout=30), 'Vd', 'Id')
            rename(f"{pathp}/IdVds - {now}.csv", f"{pathp}/IdVds - {format(Elsa.GetT('t4k'), '07.3f')} - {now}.csv")
            sleep(60)
            
        print(INO.opench(0))
        WriteLog('', path + 'log.txt')
        return 0
    #
    #
    #######################
        
    ####################### Measure CrossBridge
    #
    #
    if 'c' in device.lower():

        HP.Set2P(params['Ifmin']/params['Iffactor'], params['Ifmax']/params['Iffactor'], params['Ifpoints'], SMUN='SMU1', SMUP='SMU2', Comp=params['VComp'])
            
        now=datetime.now().strftime('%y%m%d %H%M')
        Rshort=Plot2P(HP.SingleSave(f"{pathp}/2P - {now}.csv", timeout=30))
        rename(f"{pathp}/2P - {now}.csv", f"{pathp}/2P - {format(Elsa.GetT('t4k'), '07.3f')} - {now}.csv")
        rename(f"{pathp}/2P - {now}.png", f"{pathp}/2P - {format(Elsa.GetT('t4k'), '07.3f')} - {now}.png")
            
        WriteLog(f"Rshort={format(Rshort, '.2f')}", path + 'log.txt')
        with open(f"{pathp}/RxT 2P.log", 'a') as RxT2p:
            RxT2p.write(f"{now},{format(Rshort, '.2f')}")

    if 'cb' in device.lower():
    
        HP.Set4P(params['Ifmin'], params['Ifmax'], params['Ifpoints'], Im='SMU1', Ip='SMU2', Comp=params['VComp'])
        
        now=datetime.now().strftime('%y%m%d %H%M')
        RCB=Plot2P(HP.SingleSave(f"{pathp}/4P - {now}.csv", timeout=30))
        rename(f"{pathp}/4P - {now}.csv", f"{pathp}/4P - {format(Elsa.GetT('t4k'), '07.3f')} - {now}.csv")
        rename(f"{pathp}/4P - {now}.png", f"{pathp}/4P - {format(Elsa.GetT('t4k'), '07.3f')} - {now}.png")
        
        WriteLog(f"RCB={format(RCB, '.2e')}", path + 'log.txt')
        with open(f"{pathp}/RxT 4P.log", 'a') as RxT4p:
            RxT4p.write(f"{now},{format(RCB, '.2e')}")

    now=datetime.now()
    while True:
        if (datetime.now()-now).seconds > params['min_wait']: break
        prog_bar.update(f"{datetime.now().strftime('%H:%M:%S')} Waiting minimum {params['min_wait']}s")
        sleep(0.5)
            
    print(INO.opench(0))
    WriteLog('', path + 'log.txt')
    return 0
    #
    #
    #######################

In [20]:
params = {
'Vmin' : 0,
'Vmax' : 1.5,
'Vstep' : 0.02,

'Vgmin' : 0.6,
'Vgmax' : 1.4,
'Vgstep' : 0.2,

'Vfmin' : 0,
'Vfmax' : 1.5,
'Vfstep' : 0.01,
'IComp' : 1e-3,

'Ifmin' : -5e-3,
'Ifmax' : 5e-3,
'Ifpoints' : 50,
'Iffactor' : 50,
'VComp' : 1.5,

'Vd' : 0.025,
'temp' : 3.338,
'HiPot' : False,
'min_wait' : 30
}

DeviceList = ['TP2', 'CB1', 'TN12', 'TN4', 'DN1']
# DeviceList = ['', '', '', 'TN4', '', '']
path = "C:/Users/Zucchi/Documents/Medidas/241007 - TN4 TN7 TN12 TP11/Cold/"
makedirs(path, exist_ok=True)

In [21]:
###### Cold measurement

prog_bar=display('',display_id=True)

start=datetime.now()
WriteLog(f"#  Measurement 0 - {Elsa.GetT('t4k')} - {start.strftime('%y%m%d %H%M')}\n", path+'log.txt')

for chn, device in enumerate(DeviceList):
    if device:
        TestDevice(device, chn, path, params)
WriteLog(f"Measurement 0 end - {Elsa.GetT('t4k')}. Duration: {int((datetime.now()-start).seconds/60)}:{(datetime.now()-start).seconds%60:02d}\n\n#######################\n", path + 'log.txt')

'19:06:03 Waiting minimum 30s'

#  Measurement 0 - 3.44276 - 241018 1900

Open INO: ch1

## Ch 1 TP2
Set IdxVgs
Vg=(0, -1.5, -0.02), Vd=-0.025, Ilim=0.001
Starting IdxVgs. Duration: 36 s                                    
Vth=-1.244 V
SecDer=-1.06 V
YFunc Fail
Closed

Open INO: ch2

## Ch 2 CB1
Set 2P - 12
I=(-0.0001, 0.0001), 50 Points
Starting 2P - 12. Duration: 7 s                                    
Rshort=26.59
Set 4P
I=(-0.005, 0.005), 50 Points
Starting 4P. Duration: 20 s                                    
RCB=3.82e-02
Closed

Open INO: ch3

## Ch 3 TN12
Set IdxVgs
Vg=(0, 1.5, 0.02), Vd=0.025, Ilim=0.001
Starting IdxVgs. Duration: 34 s                                    
Vth=0.535 V
SecDer=0.58 V
Yfunc 
Vth=0.52 V, SS=20.0 mV/dec, miyf=3125.6, theta1=4.387e-01, theta2=6.023e-01
003.436,0.52,20.0,3125.6,4.387e-01,6.023e-01
Closed

Open INO: ch4

## Ch 4 TN4
Set IdxVgs
Vg=(0, 1.5, 0.02), Vd=0.025, Ilim=0.001
Starting IdxVgs. Duration: 35 s                                    
Vth=0.702 V
SecDer=1.48 V
Yfunc 
Vt

In [ ]:
####### Wait PT off

while Elsa.GetPulseTubeStatus():
    sleep(30)
last_temp=4

In [45]:
###### Cooldown measurement

freq_temp=10
MsrNo=3 ## Take {MsrNo} measurements
MsrWait=1 ## wait {MsrWait} minutes

prog_bar=display('',display_id=True)

while last_temp > 280:
    current_temp=Elsa.GetT('t4k')
    
    #loop until temp changes and is a multiple of freq_temp
    while last_temp==np.around(current_temp) or current_temp%freq_temp > 0.5:
        for i in range(15):
            sleep(1)
            prog_bar.update(f"{datetime.now().strftime('%H:%M:%S')} - T = {format(current_temp, '.1f')} K")
        current_temp=Elsa.GetT('t4k')
        
    last_temp=np.around(current_temp)
    start=datetime.now()
    
    for Msr in range(MsrNo):
        plt.close('all')
        clear_output(wait=True)
        prog_bar=display('',display_id=True)
        
        WriteLog(f"# {current_temp} K - Measurement {Msr+1} - {datetime.now().strftime('%y%m%d %H%M')}\n", path+'log.txt')
        for chn, device in enumerate(DeviceList):
            if device:
                TestDevice(device, chn, path, params)
        WriteLog(f"{current_temp} K - Measurement {Msr+1} end . Duration: {int((datetime.now()-start).seconds/60)}:{(datetime.now()-start).seconds%60:02d}\n\n#######################\n", path + 'log.txt')

        current_temp=Elsa.GetT('t4k')
        for i in range(int(MsrWait*60)):
            sleep(1)
            prog_bar.update(f"{datetime.now().strftime('%H:%M:%S')} - T = {format(current_temp, '.1f')} K")

''

In [16]:
############# Cold Measurements

prog_bar=display('',display_id=True)

# for i in range(1*60):
#     sleep(1)
#     if not i%15:
#         current_temp=Elsa.GetT('t4k')
#     prog_bar.update(f"{datetime.now().strftime('%H:%M:%S')} - T = {format(current_temp, '.3f')} K")

path = "C:/Users/Zucchi/Documents/Medidas/241007 - TN4 TN7 TN12 TP11/Cold/"
makedirs(path, exist_ok=True)

MsrNo=6 ## Take {MsrNo} measurements
MsrWait=10 ## wait {MsrWait} minutes

params['HiPot']=True

for Msr in range(MsrNo):
    plt.close('all')
    clear_output(wait=True)
    prog_bar=display('',display_id=True)
    current_temp=Elsa.GetT('t4k')
    start=datetime.now()
    WriteLog(f"# {current_temp} K - Measurement {Msr+1} - {start.strftime('%y%m%d %H%M')}\n", path+'log.txt')
    for chn, device in enumerate(DeviceList):
        if device:
            TestDevice(device, chn, path, params)
    WriteLog(f"{current_temp} K - Measurement {Msr+1} end . Duration: {int((datetime.now()-start).seconds/60)}:{(datetime.now()-start).seconds%60:02d}\n\n#######################\n", path + 'log.txt')
    
    for i in range(MsrWait*60):
            sleep(1)
            prog_bar.update(f"{datetime.now().strftime('%H:%M:%S')} - T = {format(current_temp, '.1f')} K")


params['HiPot']=False

'19:08:30 - T = 3.5 K'

# 3.5343 K - Measurement 6 - 241014 1830

Open INO: ch1

## Ch 1 TP2
Set IdxVgs
Vg=(0, -1.5, -0.02), Vd=-0.025, Ilim=0.001
Starting IdxVgs. Duration: 36 s
Vth= -1.251 V
SecDer Fail
Set IdxVgs Sat
Vg=(0, -1.5, -0.02), Vd=-1.5, Ilim=0.001
Starting IdxVgs Sat. Duration: 35 s
n=70.865, Ispec=1.604e-09 A
Set VpxVgs
Is=1.604e-09, Vg=(0, -1.5, -0.05), Vlim=1.5
Starting VpxVgs. Duration: 17 s
VTO= -0.05 V
Set IdxVds
Vd=(0, -1.5, -0.02), Vg=(-0.6, -1.4, -0.2), Ilim=0.001
Starting IdxVds. Duration: 226 s0s | 30s 30s | 30s 30s | 30s 
Closed

Open INO: ch2

## Ch 2 CB1
Set 2P - 12
I=(-0.0001, 0.0001), 50 Points
Starting 2P - 12. Duration: 7 s
Rshort=26.35
Set 4P
I=(-0.005, 0.005), 50 Points
Starting 4P. Duration: 20 s
RCB=3.85e-02
Closed

Open INO: ch3

## Ch 3 TN12
Set IdxVgs
Vg=(0, 1.5, 0.02), Vd=0.025, Ilim=0.001
Starting IdxVgs. Duration: 33 s
Vth= 0.542 V
SecDer Fail
Set IdxVgs Sat
Vg=(0, 1.5, 0.02), Vd=1.5, Ilim=0.001
Starting IdxVgs Sat. Duration: 31 s
n=114.014, Ispec=5.349e-08 A
Set VpxVg

In [17]:
HP.close()
INO.close()